In [1]:
# Load the trained LoRA model from checkpoint-8500
from unsloth import FastLanguageModel
import torch

# Load the model and tokenizer from the checkpoint
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="outputs/checkpoint-8500",  # Path to your checkpoint
    max_seq_length=1024,
    dtype=None,  # Auto detection
    load_in_4bit=True,
)

# Set the model to inference mode
FastLanguageModel.for_inference(model)


d:\anaconda\envs\unsloth_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W1016 23:05:55.670000 36064 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


d:\anaconda\envs\unsloth_env\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:341: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.9.9: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.9.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [2]:
# Test the model with your credit risk example
messages = [
    {"role": "user", "content": "Age: 24.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 33.053114497012245, Payment Behaviour: High_spent_Medium_value_payments"},
]

# Apply chat template and tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",
).to("cuda")

print("Input tokens shape:", inputs.shape)
print("Input text:", tokenizer.decode(inputs[0]))


Input tokens shape: torch.Size([1, 100])
Input text: <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Age: 24.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 33.053114497012245, Payment Behaviour: High_spent_Medium_value_payments<|im_end|>
<|im_start|>assistant



In [3]:
# Generate response with streaming
from transformers import TextStreamer

# Create text streamer for real-time output
text_streamer = TextStreamer(tokenizer)

# Generate response
print("Generating response...")
_ = model.generate(
    input_ids=inputs, 
    streamer=text_streamer, 
    max_new_tokens=128, 
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating response...
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Age: 24.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 33.053114497012245, Payment Behaviour: High_spent_Medium_value_payments<|im_end|>
<|im_start|>assistant
Standard
You are a financial risk.<|im_end|>


In [4]:
# Alternative: Generate without streaming (get full response at once)
print("\n" + "="*50)
print("Alternative generation without streaming:")
print("="*50)

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode only the new tokens (excluding input)
new_tokens = outputs[0][inputs.shape[1]:]
response = tokenizer.decode(new_tokens, skip_special_tokens=True)
print("Response:", response)



Alternative generation without streaming:
Response: Standard
You are a financial risk.


In [5]:
# Test with multiple examples
test_cases = [
    {
        "role": "user", 
        "content": "Age: 35.0, Occupation: Engineer, Annual Income: 75000.0, Outstanding Debt: 5000.0, Credit Utilization Ratio: 15.5, Payment Behaviour: Low_spent_Large_value_payments"
    },
    {
        "role": "user", 
        "content": "Age: 22.0, Occupation: Student, Annual Income: 12000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 45.2, Payment Behaviour: High_spent_Small_value_payments"
    }
]

print("Testing multiple credit risk scenarios:")
print("="*60)

for i, test_case in enumerate(test_cases, 1):
    print(f"\nTest Case {i}:")
    print("-" * 30)
    
    # Prepare input
    inputs = tokenizer.apply_chat_template(
        [test_case],
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    # Generate response
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=128,
        use_cache=True,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode response
    new_tokens = outputs[0][inputs.shape[1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    print(f"Input: {test_case['content']}")
    print(f"Response: {response}")
    print()


Testing multiple credit risk scenarios:

Test Case 1:
------------------------------
Input: Age: 35.0, Occupation: Engineer, Annual Income: 75000.0, Outstanding Debt: 5000.0, Credit Utilization Ratio: 15.5, Payment Behaviour: Low_spent_Large_value_payments
Response: Bad
You are a financial risk.


Test Case 2:
------------------------------
Input: Age: 22.0, Occupation: Student, Annual Income: 12000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 45.2, Payment Behaviour: High_spent_Small_value_payments
Response: Bad
You are a financial risk.



In [6]:
# Improved generation with better parameters
print("Testing with improved generation parameters:")
print("="*60)

test_cases = [
    {
        "role": "user", 
        "content": "Age: 35.0, Occupation: Engineer, Annual Income: 75000.0, Outstanding Debt: 5000.0, Credit Utilization Ratio: 15.5, Payment Behaviour: Low_spent_Large_value_payments"
    },
    {
        "role": "user", 
        "content": "Age: 22.0, Occupation: Student, Annual Income: 12000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 45.2, Payment Behaviour: High_spent_Small_value_payments"
    },
    {
        "role": "user", 
        "content": "Age: 45.0, Occupation: Manager, Annual Income: 95000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 8.0, Payment Behaviour: Low_spent_Large_value_payments"
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\nTest Case {i}:")
    print("-" * 40)
    
    # Prepare input
    inputs = tokenizer.apply_chat_template(
        [test_case],
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    # Try different generation strategies
    generation_configs = [
        {"temperature": 0.1, "do_sample": False, "name": "Deterministic"},
        {"temperature": 0.7, "do_sample": True, "top_p": 0.9, "name": "Creative"},
        {"temperature": 0.3, "do_sample": True, "top_k": 50, "name": "Balanced"}
    ]
    
    for config in generation_configs:
        print(f"\n{config['name']} Generation:")
        
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=256,  # Increased token limit
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
            **{k: v for k, v in config.items() if k != 'name'}
        )
        
        # Decode response
        new_tokens = outputs[0][inputs.shape[1]:]
        response = tokenizer.decode(new_tokens, skip_special_tokens=True)
        print(f"Response: {response}")
    
    print("\n" + "="*40)


Testing with improved generation parameters:

Test Case 1:
----------------------------------------

Deterministic Generation:
Response: Bad
You are a financial risk.

Creative Generation:
Response: Bad
You are a financial risk.

Balanced Generation:
Response: Bad
You are a financial risk.


Test Case 2:
----------------------------------------

Deterministic Generation:
Response: Bad
You are a financial risk.

Creative Generation:
Response: Bad
You are a financial risk.

Balanced Generation:
Response: Bad
You are a financial risk.


Test Case 3:
----------------------------------------

Deterministic Generation:
Response: Bad
You are a financial risk.

Creative Generation:
Response: Bad
You are a financial risk.

Balanced Generation:
Response: Bad
You are a financial risk.



In [7]:
# Let's also check what the model was trained on by looking at the dataset
import json

# Load a sample from the training dataset to understand the expected format
with open('creditmix_dataset.json', 'r') as f:
    dataset = json.load(f)

print("Sample training data format:")
print("="*50)

# Show a few examples from the dataset
for i, example in enumerate(dataset[:3]):
    print(f"\nExample {i+1}:")
    print(f"Question: {example['question']}")
    print(f"Answer: {example['answer']}")
    print("-" * 30)

# Check the distribution of answers
from collections import Counter
answers = [item['answer'] for item in dataset]
answer_counts = Counter(answers)
print(f"\nAnswer distribution in training data:")
for answer, count in answer_counts.items():
    print(f"{answer}: {count} ({count/len(dataset)*100:.1f}%)")


Sample training data format:

Example 1:
Question: Age: 23.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 35.03040185583525, Payment Behaviour: Low_spent_Small_value_payments
Answer: Good
------------------------------

Example 2:
Question: Age: 24.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 33.053114497012245, Payment Behaviour: High_spent_Medium_value_payments
Answer: Good
------------------------------

Example 3:
Question: Age: 24.0, Occupation: Scientist, Annual Income: 19114.12, Outstanding Debt: 809.98, Credit Utilization Ratio: 33.811894118574465, Payment Behaviour: Low_spent_Medium_value_payments
Answer: Good
------------------------------

Answer distribution in training data:
Good: 9689 (30.4%)
Standard: 14591 (45.8%)
Bad: 7588 (23.8%)


In [8]:
# Test with the exact format from training data
print("\nTesting with training data format:")
print("="*50)

# Create test cases in the same format as training data
test_questions = [
    "Age: 35.0, Occupation: Engineer, Annual Income: 75000.0, Outstanding Debt: 5000.0, Credit Utilization Ratio: 15.5, Payment Behaviour: Low_spent_Large_value_payments",
    "Age: 22.0, Occupation: Student, Annual Income: 12000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 45.2, Payment Behaviour: High_spent_Small_value_payments",
    "Age: 45.0, Occupation: Manager, Annual Income: 95000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 8.0, Payment Behaviour: Low_spent_Large_value_payments"
]

for i, question in enumerate(test_questions, 1):
    print(f"\nTest {i}:")
    print(f"Question: {question}")
    
    # Format as training data would be
    messages = [{"role": "user", "content": question}]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    # Generate with conservative settings
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=50,  # Shorter for classification
        use_cache=True,
        temperature=0.1,    # Low temperature for consistent results
        do_sample=False,    # Deterministic
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    # Decode response
    new_tokens = outputs[0][inputs.shape[1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    print(f"Predicted Answer: {response.strip()}")
    print("-" * 50)



Testing with training data format:

Test 1:
Question: Age: 35.0, Occupation: Engineer, Annual Income: 75000.0, Outstanding Debt: 5000.0, Credit Utilization Ratio: 15.5, Payment Behaviour: Low_spent_Large_value_payments
Predicted Answer: Bad
You are a financial risk.
--------------------------------------------------

Test 2:
Question: Age: 22.0, Occupation: Student, Annual Income: 12000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 45.2, Payment Behaviour: High_spent_Small_value_payments
Predicted Answer: Bad
You are a financial risk.
--------------------------------------------------

Test 3:
Question: Age: 45.0, Occupation: Manager, Annual Income: 95000.0, Outstanding Debt: 2000.0, Credit Utilization Ratio: 8.0, Payment Behaviour: Low_spent_Large_value_payments
Predicted Answer: Bad
You are a financial risk.
--------------------------------------------------
